The idea:
- Take in a description of the ticket as well as the issue type

Imports

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import psycopg2
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack, csr_matrix


In [26]:
conn = psycopg2.connect(
    dbname="postgres", 
    user="root", 
    password="password1", 
    host="localhost", 
    port="5432"
)

# Define your SQL query
query = 'SELECT * FROM "Issue"'


# Execute the query and load the result into a DataFrame
df = pd.read_sql(query, conn)
# print(df.head())


# Define the column mapping
column_mapping = {
    "Summary": "summary",
    "Issue key": "issueKey",
    "Issue id": "issueId",
    "Issue Type": "issueType",
    "Status": "status",
    "Project key": "projectKey",
    "Project name": "projectName",
    "Priority": "priority",
    "Resolution": "resolution",
    "Assignee": "assignee",
    "Reporter (Email)": "reporterEmail",
    "Creator (Email)": "creatorEmail",
    "Created": "created",
    "Updated": "updated",
    "Last Viewed": "lastViewed",
    "Resolved": "resolved",
    "Due date": "dueDate",
    "Description": "description",
    "Partner Names": "partnerNames",
    "Custom field (Cause of issue)": "causeOfIssue",
    "Custom field (Record/Transaction ID)": "recordTransactionId",
    "Custom field (Region)": "region",
    "Custom field (Relevant Departments)": "relevantDepartments",
    "Custom field (Relevant Departments).1": "relevantDepartments1",
    "Custom field (Request Category)": "requestCategory",
    "Custom field (Request Type)": "requestType",
    "Custom field (Request language)": "requestLanguage",
    "Custom field (Resolution Action)": "resolutionAction",
    "Satisfaction rating": "satisfactionRating",
    "Custom field (Satisfaction date)": "satisfactionDate",
    "Custom field (Source)": "source",
    "Custom field (Time to first response)": "timeToFirstResponse",
    "Custom field (Time to resolution)": "timeToResolution",
    "Custom field (Work category)": "workCategory",
    "Status Category": "statusCategory",
    "Status Category Changed": "statusCategoryChanged",
    "Custom field ([CHART] Date of First Response)": "dateOfFirstResponse",
    "comments": "comments"
}

# Rename the columns in the DataFrame
df.rename(columns=column_mapping, inplace=True)


/tmp/ipykernel_16642/3375834500.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [9]:
print(df["causeOfIssue"].unique())

print(df["requestType"].unique())

print(df["requestCategory"].unique())

['' 'Request of Customer Relations' 'Other' 'Partner Error' 'Ops Issue'
 'Partner Knowledge/Training' 'Menu Discrepancy' 'System Error']
['General Questions' 'Email Request' 'Website Assistance (PartnerLink)'
 'Pickup & Delivery Support' 'Request New Shopper Access'
 'Revise My Order' 'Request Produce' 'Grant Support' 'Billing Support'
 'Provide Menu Feedback' 'Request a Return or Refund'
 'Questions about Shopping Menu Items'
 'Provide Operations/Transportation Feedback']
['' 'Orders - Pre delivery -> Cancelation'
 'Orders - Pre delivery -> Produce Request'
 'Orders - Pre delivery -> General Questions'
 'Orders - Pre delivery -> Edit Order Items'
 'Billing / Grants -> Grant Support'
 'Agency Administration -> Connection to CAFB Staff'
 'Orders - Pre delivery' 'Delivery / Pickup -> Return - Quality'
 'PartnerLink -> New Shopper' 'Delivery / Pickup -> General Questions'
 'Orders - Pre delivery -> Report Issue'
 'Orders - Pre delivery ->  Date/Time Change - New window'
 'Delivery / Picku

In [27]:
def classify_ticket(ticket_data, model, vectorizer, label_encoders):
    """
    Uses an ML model to classify a ticket based on multiple fields.
    :param ticket_data: A dictionary with ticket fields.
    :param model: The trained classification model.
    :param vectorizer: The vectorizer for text transformation.
    :param label_encoders: Dictionary of label encoders for categorical fields.
    :return: The predicted category of the ticket.
    """
    text_features = vectorizer.transform([ticket_data["Summary"] + " " + ticket_data["Description"]])
    cat_features = hstack([
        label_encoders["causeOfIssue"].transform([ticket_data["causeOfIssue"]]).reshape(-1, 1),
        label_encoders["requestType"].transform([ticket_data["requestType"]]).reshape(-1, 1)
    ])
    features = hstack([text_features, cat_features])
    return model.predict(features)[0]

In [28]:
label_encoders = {col: LabelEncoder().fit(df[col]) for col in ["causeOfIssue", "requestType"]}
df["causeOfIssue"] = label_encoders["causeOfIssue"].transform(df["causeOfIssue"])
df["requestType"] = label_encoders["requestType"].transform(df["requestType"])

# Vectorize text fields
vectorizer = TfidfVectorizer()
X_text = vectorizer.fit_transform(df["summary"] + " " + df["description"])
X_cat = csr_matrix(df[["causeOfIssue", "requestType"]].values)
X = hstack([X_text, X_cat])
df["category"] = None
y = ["Add to Order", "Cancel Order", "Change Order", "Check Status", "Refund Request"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)

# Example Usage
ticket_example = {
    "Summary": "Modify my order details",
    "Description": "I need to update my delivery address.",
    "causeOfIssue": "Address Change",
    "requestType": "Change Order"
}

print(f"Predicted Category: {classify_ticket(ticket_example, model, vectorizer, label_encoders)}")

ValueError: Found input variables with inconsistent numbers of samples: [1000, 5]